<a href="https://colab.research.google.com/github/Taichi2005/GoogleColab_Whisper/blob/main/%E3%80%90%E5%AE%8C%E6%88%90%E7%89%88%E3%80%91%E5%8B%95%E7%94%BBURL%E3%81%8B%E3%82%89%E9%AB%98%E7%B2%BE%E5%BA%A6%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97%E5%AE%9F%E8%A1%8C%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### **【完成版】動画URLから高精度文字起こし実行スクリプト**

#以下の3つのセルを上から順番に実行してください。

#### **セル1: 環境構築**
#このセルでは、文字起こしに必要なライブラリやツールをすべてインストールします。初回実行時は数分かかることがあります。

# セル1: 環境構築
# -------------------------------------------------------------------------------------------
# GPUが有効になっているかを確認
print("▼ GPUの確認")
!nvidia-smi

# 必要なライブラリとツールをインストール
# yt-dlp: 様々なサイトから動画や音声をダウンロードするツール
# faster-whisper: 高速化されたWhisperモデル
# ffmpeg: 動画・音声の処理・変換に必須のツール
print("\n▼ 必要なライブラリとツールのインストール")
!pip install -U yt-dlp -q
!pip install faster-whisper==1.0.3 -q
!pip install nvidia-cublas-cu12==12.6.4.1 -q
!pip install nvidia-cudnn-cu12==9.10.2.21 -q
!apt-get update && apt-get install -y ffmpeg -qq

print("\n✅ 環境構築が完了しました。")
print("次のセルに進んでGoogle Driveに接続してください。")

▼ GPUの確認
Sat Oct 25 13:31:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------------------

In [2]:
#### **セル2: Google Driveへの接続 (任意)**
#文字起こししたテキストファイルをGoogle Driveに保存したい場合に、このセルを実行してGoogle Driveをマウント（接続）してください。

# -------------------------------------------------------------------------------------------
# セル2: Google Driveへの接続
# -------------------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Driveへの接続が完了しました。")
print("最後のセルに進んで、文字起こしを実行してください。")

Mounted at /content/drive

✅ Google Driveへの接続が完了しました。
最後のセルに進んで、文字起こしを実行してください。


In [5]:
#### **セル3: 🚀 URLから高精度文字起こし実行**
#ここに動画のURLを入力し、設定を調整してから実行ボタン（▶）を押してください。

#@title 🚀 URLから高精度文字起こし実行
#@markdown ### 1. 動画のURLと出力先の設定
#@markdown ---
#@markdown 文字起こししたい動画のURLを入力してください。
video_url = "" #@param {type:"string"}

#@markdown 結果（.txtファイル）を保存するフォルダのパスを指定します。
#@markdown **ローカルに保存する場合:** `/content/transcripts` のように入力します。
#@markdown **Google Driveに保存する場合:** `/content/drive/MyDrive/transcripts` のように入力します。（セル2の実行が必要です）
output_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts" #@param {type:"string"}

#@markdown ### 2. モデルとパフォーマンス設定
#@markdown ---
#@markdown **モデル**: `Zoont/...-int8-ct2`は精度を維持しつつ最速・省メモリな**総合推奨モデル**です。
model_name = "Zoont/faster-whisper-large-v3-turbo-int8-ct2" #@param ["Zoont/faster-whisper-large-v3-turbo-int8-ct2", "deepdml/faster-whisper-large-v3-turbo-ct2", "large-v3", "large-v2", "distil-large-v3", "medium", "small", "base", "tiny"]
#@markdown **計算タイプ**: `int8`モデルには`int8_float16`、`float16`モデルには`float16`の組み合わせを推奨します。
compute_type = "int8_float16" #@param ["int8_float16", "float16", "int8", "float32"]

#@markdown ### 3. VAD (音声区間検出) 設定
#@markdown ---
#@markdown VADを有効にすると、無音区間を自動で除去し、文字起こしの精度を向上させることができます。
use_vad_filter = True #@param {type:"boolean"}
#@markdown ここで指定したミリ秒以上の無音を「発話の区切り」とみなします。
vad_min_silence_duration_ms = 500 #@param {type:"slider", min:100, max:2000, step:100}

#@markdown ### 4. 高度な設定（オプション）
#@markdown ---
#@markdown **ビームサイズ (beam_size)**: 文字起こしの精度と速度のトレードオフを調整します。値を大きくすると精度が向上する可能性がありますが、処理速度は低下します。`faster-whisper`では`5`がバランスの取れた推奨値です。
beam_size = 5 #@param {type:"slider", min:1, max:10, step:1}

#@markdown **中間ファイルのクリーンアップ**: チェックを入れると、処理終了後にダウンロードした音声ファイルを自動で削除します。
cleanup_audio_file = True #@param {type:"boolean"}


# --- ここから下は実行コード（編集不要） ---
import os
import yt_dlp
from faster_whisper import WhisperModel
import time
from datetime import datetime, timezone, timedelta
import torch

def get_current_timestamp():
    """現在時刻を[YYYY-MM-DD HH:MM:SS]形式の文字列で返す"""
    JST = timezone(timedelta(hours=+9))
    return datetime.now(JST).strftime('%Y-%m-%d %H:%M:%S')

def run_pipeline():
    """メインの処理パイプライン"""
    if not video_url:
        print("❌ エラー: 動画のURLが入力されていません。")
        return

    print(f"{get_current_timestamp()} --- 1. パイプライン開始 ---")

    # 出力ディレクトリの作成
    os.makedirs(output_dir, exist_ok=True)
    print(f"出力先フォルダ: {output_dir}")

    # 一時ダウンロード用ディレクトリの作成
    temp_audio_dir = "/content/temp_audio"
    os.makedirs(temp_audio_dir, exist_ok=True)

    # 2. モデルのロード
    print(f"\n{get_current_timestamp()} --- 2. モデル '{model_name}' ({compute_type}) をロード中... ---")
    start_load_time = time.time()
    try:
        model = WhisperModel(model_name, device="cuda", compute_type=compute_type)
    except Exception as e:
        print(f"❌ モデルのロード中にエラーが発生しました: {e}")
        print("   -> ランタイムのタイプがGPUになっているか確認してください。")
        return

    end_load_time = time.time()
    print(f"✅ モデルのロードが完了しました。({end_load_time - start_load_time:.2f}秒)")

    # 3. yt-dlpによる音声のダウンロードと抽出
    print(f"\n{get_current_timestamp()} --- 3. URLから音声のダウンロードと抽出を開始 ---")
    print(f"対象URL: {video_url}")

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav', # Whisperが最も得意とするWAV形式に変換
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(temp_audio_dir, '%(id)s.%(ext)s'),
        'quiet': True,
    }

    audio_file_path = None
    video_title = "untitled"
    video_id = "default_id"
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=True)
            video_id = info.get('id', 'default_id')
            video_title = info.get('title', 'untitled')
            # 拡張子を.wavに変更したパスを生成
            base, _ = os.path.splitext(ydl.prepare_filename(info))
            audio_file_path = base + ".wav"

        if not os.path.exists(audio_file_path):
            raise FileNotFoundError("音声ファイルの生成に失敗しました。")

        print(f"✅ 音声ファイルの準備が完了しました: {os.path.basename(audio_file_path)}")
        print(f"   動画タイトル: {video_title}")

    except Exception as e:
        print(f"❌ 音声のダウンロードまたは変換中にエラーが発生しました: {e}")
        return

    # 4. 文字起こし処理
    print(f"\n{get_current_timestamp()} --- 4. 文字起こし処理開始 ---")
    start_transcribe_time = time.time()
    try:
        print(f"  - 設定 (beam_size: {beam_size}, VAD: {'有効' if use_vad_filter else '無効'})")
        segments, info = model.transcribe(
            audio_file_path,
            beam_size=beam_size,
            vad_filter=use_vad_filter,
            vad_parameters=dict(min_silence_duration_ms=vad_min_silence_duration_ms)
        )
        print(f"  - 検出言語: {info.language} (確率: {info.language_probability:.2f})")

        # 5. 結果の保存
        output_txt_filename = f"{video_id}_{video_title[:50]}.txt".replace('/','-') # ファイル名に使えない文字を置換
        drive_output_path = os.path.join(output_dir, output_txt_filename)

        with open(drive_output_path, "w", encoding="utf-8") as f:
            f.write(f"動画タイトル: {video_title}\n")
            f.write(f"URL: {video_url}\n")
            f.write(f"文字起こし実行日時: {get_current_timestamp()}\n")
            f.write(f"モデル: {model_name} ({compute_type})\n\n---\n\n")

            full_text = ""
            for segment in segments:
                line = f"[{segment.start:0>7.2f}s -> {segment.end:0>7.2f}s] {segment.text.strip()}\n"
                f.write(line)
                full_text += segment.text.strip() + " "

        end_transcribe_time = time.time()
        print(f"✅ 文字起こしが完了しました。({end_transcribe_time - start_transcribe_time:.2f}秒)")
        print(f"   -> 結果を保存しました: {drive_output_path}")

        print("\n--- 書き起こし結果 (冒頭) ---")
        print(full_text[:500] + "...")
        print("--------------------------")

    except Exception as e:
        print(f"❌ 文字起こし中にエラーが発生しました: {e}")
    finally:
        # 6. 中間ファイルのクリーンアップ
        if cleanup_audio_file and audio_file_path and os.path.exists(audio_file_path):
            os.remove(audio_file_path)
            print(f"\n{get_current_timestamp()} --- 5. 中間音声ファイルを削除しました ---")

    print(f"\n{get_current_timestamp()} --- 🎉 全ての処理が完了しました ---")

# パイプライン実行
run_pipeline()

2025-10-25 22:38:06 --- 1. パイプライン開始 ---
出力先フォルダ: /content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts

2025-10-25 22:38:10 --- 2. モデル 'Zoont/faster-whisper-large-v3-turbo-int8-ct2' (int8_float16) をロード中... ---
✅ モデルのロードが完了しました。(0.91秒)

2025-10-25 22:38:11 --- 3. URLから音声のダウンロードと抽出を開始 ---
対象URL: https://youtu.be/M3AezqTHS2o?si=e4e3OSMcK1B0skRg


         player = https://www.youtube.com/s/player/6e4dbefe/player_ias.vflset/en_US/base.js
         n = PVg62htio8WDXOR7 ; player = https://www.youtube.com/s/player/6e4dbefe/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


✅ 音声ファイルの準備が完了しました: M3AezqTHS2o.wav
   動画タイトル: 【談合】マンション大規模修繕で理事へのキックバック横行？阻止法はあるのか

2025-10-25 22:38:28 --- 4. 文字起こし処理開始 ---
  - 設定 (beam_size: 5, VAD: 有効)
  - 検出言語: ja (確率: 1.00)
✅ 文字起こしが完了しました。(86.55秒)
   -> 結果を保存しました: /content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts/M3AezqTHS2o_【談合】マンション大規模修繕で理事へのキックバック横行？阻止法はあるのか.txt

--- 書き起こし結果 (冒頭) ---
マンションを購入してただいま10年目ぐらいなんですが レジネンスのマンションですよね 住むための 中古で今はもう地区30年を超えている状態でして そろそろ大規模修繕3回目をやろうかという話になりまして その中で自分の方が大規模修繕委員に専任されまして ただ過去の大規模修繕の話とかを聞くと 結構1度目と2度目で方式とかがガラッと変わってしまっておりまして 過去の修繕した方も担当もいらっしゃらないということで 担当というのは管理組合の中にいない マンションの修繕委員の経験がいない それに加えて結構いろんなニュースが最近大規模修繕でありましたので ちょっと不安になりましていつも見させていただいていたりします マンションで言いますと結構50個ぐらいの小さいマンションになっておりまして もう本当30年経って1回目2回目と修繕をやっているんですが 1度目がどうやら設計管理方式といって いろいろ業者さんをマンション設計士さんから経由して 何でしょう 協売って言うんですかね 入札みたいな 入札して選んだ状態と 2回目はもともと管理している 建設の管理会社 というところは主導で そこに任せよという形にな...
--------------------------

2025-10-25 22:39:55 --- 5. 中間音声ファイルを削除しました ---

2025-10-25 22:39:55 --- 🎉 全ての処理が完了しました ---
